# Question 1

Discuss the concept and application of Time series analysis using machine learning modelling by providing a real-world data set. 

What is the purpose of The Augmented Dickey-Fuller test in time series?
Augmented Dickey Fuller test (ADF Test) is a common statistical test used to test whether a given Time series is stationary or not. It is one of the most commonly used statistical test when it comes to analyzing the stationary of a series. The ARIMA model mostly fit on the stationary dataset so we check the dataset is stationary or not by using the Dickey-Fuller Test.

a) Apply an appropriate Box-Jenkins model to the chosen dataset (ARMA, ARIMA, SARIMA etc). Check for the model adequacy.


In [1]:
## Import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

In [2]:
## Import dataset
df=pd.read_csv('jewelry.csv')
df.head()

,2018-12-01 11:40:29 UTC,1924719191579951782,1842195256808833386,1,1806829201890738522,jewelry.earring,0,561.51,1515915625207851155,Unnamed: 9,red,gold,diamond
0,2018-12-01 17:38:31 UTC,1924899396621697920,1806829193678291446,1,1.806829e+18,NaN,NaN,212.14,1.515916e+18,NaN,yellow,gold,NaN
1,2018-12-02 13:53:42 UTC,1925511016616034733,1842214461889315556,1,1.806829e+18,jewelry.pendant,1.0,54.66,1.515916e+18,f,white,gold,sapphire
2,2018-12-02 17:44:02 UTC,1925626951238681511,1835566849434059453,1,1.806829e+18,jewelry.pendant,0.0,88.90,1.515916e+18,f,red,gold,diamond
3,2018-12-02 21:30:19 UTC,1925740842841014667,1873936840742928865,1,1.806829e+18,jewelry.necklace,0.0,417.67,1.515916e+18,NaN,red,gold,amethyst
4,2018-12-02 22:09:34 UTC,1925760595336888995,1835566854827934449,1,1.806829e+18,jewelry.earring,1.0,102.27,1.515916e+18,NaN,red,gold,NaN


In [3]:
df.isnull().sum()

2018-12-01 11:40:29 UTC        0
1924719191579951782            0
1842195256808833386            0
1                              0
1806829201890738522         5352
jewelry.earring             9933
0                           4785
561.51                      5352
1515915625207851155         5352
Unnamed: 9                 48167
red                         7660
gold                        5462
diamond                    34058
dtype: int64

In [4]:
# Checking dataset dimension
df.shape

(95910, 13)

In [26]:
# check dataset column type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95910 entries, 0 to 95909
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   order_date   95910 non-null  object 
 1   order_id     95910 non-null  int64  
 2   user_id      95910 non-null  int64  
 3   purchase     95910 non-null  int64  
 4   category_id  95910 non-null  float64
 5   category     85977 non-null  object 
 6   brand        91125 non-null  float64
 7   price        90558 non-null  float64
 8   price_id     90558 non-null  float64
 9   buyer        47743 non-null  object 
 10  colour1      88250 non-null  object 
 11  colour2      90448 non-null  object 
 12  type         61852 non-null  object 
dtypes: float64(4), int64(3), object(6)
memory usage: 9.5+ MB


In [6]:
df.head()
#no columns, add columns
columns = ['order_date','order_id','user_id','purchase','category_id','category','brand','price','price_id','buyer','colour1','colour2','type']
df.columns = columns

In [7]:
print(df.head())

                order_date             order_id              user_id  \
0  2018-12-01 17:38:31 UTC  1924899396621697920  1806829193678291446   
1  2018-12-02 13:53:42 UTC  1925511016616034733  1842214461889315556   
2  2018-12-02 17:44:02 UTC  1925626951238681511  1835566849434059453   
3  2018-12-02 21:30:19 UTC  1925740842841014667  1873936840742928865   
4  2018-12-02 22:09:34 UTC  1925760595336888995  1835566854827934449   

   purchase   category_id          category  brand   price      price_id  \
0         1  1.806829e+18               NaN    NaN  212.14  1.515916e+18   
1         1  1.806829e+18   jewelry.pendant    1.0   54.66  1.515916e+18   
2         1  1.806829e+18   jewelry.pendant    0.0   88.90  1.515916e+18   
3         1  1.806829e+18  jewelry.necklace    0.0  417.67  1.515916e+18   
4         1  1.806829e+18   jewelry.earring    1.0  102.27  1.515916e+18   

  buyer colour1 colour2      type  
0   NaN  yellow    gold       NaN  
1     f   white    gold  sapphire  
2 

In [41]:
df.isnull().sum()

order_date        0
user_id           0
purchase          0
category       9933
brand          4785
price          5352
buyer         48167
type          34058
dtype: int64

In [38]:
#drop the ids, not needed
df.drop(columns=['order_id','category_id','price_id','colour1','colour2'],inplace=True)

In [43]:
unique_entries_dataframe = df.apply(lambda x: x.unique())
print(unique_entries_dataframe)

order_date    [2018-12-01 17:38:31 UTC, 2018-12-02 13:53:42 ...
user_id       [1806829193678291446, 1842214461889315556, 183...
purchase                                                    [1]
category      [nan, jewelry.pendant, jewelry.necklace, jewel...
brand         [nan, 1.0, 0.0, 2.0, 4.0, 6.0, 5.0, 3.0, 1.550...
price         [212.14, 54.66, 88.9, 417.67, 102.27, 150.55, ...
buyer                                               [nan, f, m]
type          [nan, sapphire, diamond, amethyst, fianit, pea...
dtype: object


In [17]:
df['category_id'].fillna(df['category_id'].mean(), inplace=True)


In [10]:
df.head()

,order_date,order_id,user_id,purchase,category_id,category,brand,price,price_id,buyer,colour1,colour2,type
0,2018-12-01 17:38:31 UTC,1924899396621697920,1806829193678291446,1,1.806829e+18,NaN,NaN,212.14,1.515916e+18,NaN,yellow,gold,NaN
1,2018-12-02 13:53:42 UTC,1925511016616034733,1842214461889315556,1,1.806829e+18,jewelry.pendant,1.0,54.66,1.515916e+18,f,white,gold,sapphire
2,2018-12-02 17:44:02 UTC,1925626951238681511,1835566849434059453,1,1.806829e+18,jewelry.pendant,0.0,88.90,1.515916e+18,f,red,gold,diamond
3,2018-12-02 21:30:19 UTC,1925740842841014667,1873936840742928865,1,1.806829e+18,jewelry.necklace,0.0,417.67,1.515916e+18,NaN,red,gold,amethyst
4,2018-12-02 22:09:34 UTC,1925760595336888995,1835566854827934449,1,1.806829e+18,jewelry.earring,1.0,102.27,1.515916e+18,NaN,red,gold,NaN


In [ ]:
## our data column should be in datetime format, so we convert this in date format
from datetime import datetime
df['InvoiceDate']=pd.to_datetime(df['InvoiceDate'])

In [ ]:
# Distribution graphs (histogram/bar graph) of column data
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()

In [ ]:
plotPerColumnDistribution(df, 10, 5)

In [ ]:
# Correlation matrix
def plotCorrelationMatrix(df, graphWidth):
    filename = df.dataframeName
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()

In [ ]:
plotCorrelationMatrix(df, 8)